# Data handling with Dataframe

Kotlin [DataFrame](https://github.com/Kotlin/dataframe) was largely inspired by pandas structures, but despite this, it has his own characteristics that make Data Analysis very understandable and concise. Most of it's readability comes from functional languages chains of transformations (the data pipeline), and the use of DSL, that makes it's syntax closer to natural language.

Dataframe is built-in kotlin jupyter kernel, so the magic command `%use` will the whole library for us

In [1]:
%use dataframe

For creating a dataframe from scratch, we can follow several approaches:

1. Creating `Column` objects for storing data, and assign columns to a `DataFrame`

In [2]:
val names by columnOf("foo", "bar", "baz")
val numbers by columnOf(1, 2, 3)

val df = dataFrameOf(names, numbers)
df

DataFrame: rowsCount = 3, columnsCount = 2

2. Providing a series of `Pair<String, Any>` or a `Map<String, Any>`

In [3]:
val df = dataFrameOf(
    "names" to listOf("foo", "bar", "baz"),
    "numbers" to listOf(1, 2, 3)
)

df

DataFrame: rowsCount = 3, columnsCount = 2

3. Loading a dataframe from a File (See references at [Loading a Dataframe from a File](#))